# Assignment 1

This assignment will involve the creation of a spellchecking system and an evaluation of its performance. You may use the code snippets provided in Python for completing this or you may use the programming language or environment of your choice

Please start by downloading the corpus `holbrook.txt` from Blackboard

The file consists of lines of text, with one sentence per line. Errors in the line are marked with a `|` as follows

    My siter|sister go|goes to Tonbury .
    
In this case the word 'siter' was corrected to 'sister' and the word 'go' was corrected to 'goes'.

In some places in the corpus two words maybe corrected to a single word or one word to a multiple words. This is denoted in the data using underscores e.g.,

    My Mum goes out some_times|sometimes .
    
For the purpose of this assignment you do not need to separate these words, but instead you may treat them like a single token.

*Note: you may use any functions from NLTK to complete the assignment. It should not be necessary to use other libraries and so please consult with us if your solution involves any other external library. If you use any function from NLTK in Task 6 please include a brief description of this function and how it contributes to your solution.*

## Task 1 (10 Marks)

Write a parser that can read all the lines of the file `holbrook.txt` and print out for each line the original (misspelled) text, the corrected text and the indexes of any changes. The indexes refers to the index of the words in the sentence. In the example given, there is only an error in the 10th word and so the list of indexes is [9]. It is not necessary to analyze where the error occurs inside the word.

Then split your data into a test set of 100 lines and a training set.

In [67]:
import nltk

lines = []  
with open('holbrook.txt', 'r',encoding='utf-8') as f:
    lines = f.readlines()
data = []

count = 0
for line in lines:
    original = []
    corrected = []
    indexes = []
    l = line.split()
    #print(l)
    for word in l:
        #print(word)
        if '|' in word:
            var = word.split('|')
            original.append(var[0])
            corrected.append(var[1])
            indexes.append(count)
        else:
            original.append(word)
            corrected.append(word)
        count += 1
    data.append({'original':original, 'corrected':corrected, 'indexes':indexes})

print(data[2])    
    

# Write your code here

#assert(data[2] == {
#    'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 
#    'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 
#    'indexes': [9]
#})

{'original': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'siter', '.'], 'corrected': ['I', 'have', 'four', 'in', 'my', 'Family', 'Dad', 'Mum', 'and', 'sister', '.'], 'indexes': [15]}


The counts and assertions given in the following sections are based on splitting the training and test set as follows

In [ ]:
test = data[:100]
train = data[100:]

## **Task 2** (10 Marks): 
Calculate the frequency (number of occurrences), *ignoring case*, of all words and their unigram probability from the corrected *training* sentences.

*Hint: use `Counter` to implement this so it may be called many times*

In [78]:
from collections import Counter

import itertools
def seperation():
    test = data[:100]
    train = data[100:]
    train_corrected = [i['corrected'] for i in train]
    train_tokenize = (list(itertools.chain.from_iterable(train_corrected)))
    return train_tokenize


def unigram(word):
    train_tokenize = seperation()
    counter = Counter(train_tokenize)
    ans = counter[f"{word}"]
    return ans
     

def prob(word):
    train_tokenize = seperation()
    length = len(train_tokenize)
    Prob = unigram(word)/length
    return Prob

prob('me')


# Test your code with the following
#assert(unigram("me")==87)

0.004031697483664674

## **Task 3** (15 Marks): 
[Edit distance](https://en.wikipedia.org/wiki/Edit_distance) is a method that calculates how similar two strings are to one another by counting the minimum number of operations required to transform one string into the other. There is a built-in implementation in NLTK that works as follows:


In [30]:
from nltk.metrics.distance import edit_distance

# Edit distance returns the number of changes to transform one word to another
print(edit_distance("hello", "hi"))

4


Write a function that calculates all words with *minimal* edit distance to the misspelled word. You should do this as follows

1. Collect the set of all unique tokens in `train`
2. Find the minimal edit distance, that is the lowest value for the function `edit_distance` between `token` and a word in `train`
3. Output all unique words in `train` that have this same (minimal) `edit_distance` value

*Do not implement edit distance, use the built-in NLTK function `edit_distance`*

In [160]:

def get_candidates(token):
    train_tokenize = seperation()
    unique = set(train_tokenize) 
    dis=[]
    for i in unique:
        dis.append([edit_distance(i,token)])

    distance = dict(zip(unique,dis))
    sorted_dict = {k: v for k, v in sorted(distance.items(), key=lambda item: item[1])}
    values = min(sorted_dict.values())
    result = [key for key in sorted_dict if sorted_dict[key]==values]
    return result  
get_candidates('48')
        
# Test your code as follows
#assert get_candidates("minde") == ['mine', 'mind']

['4', '18', '8', '40']

## Task 4 (15 Marks):

Write a function that takes a (misspelled) sentence and returns the corrected version of that sentence. The system should scan the sentence for words that are not in the dictionary (set of unique words in the training set) and for each word that is not in the dictionary choose a word in the dictionary that has minimal edit distance and has the highest *unigram probability*. 

*Your solution to this should involve `get_candidates`*


In [171]:
def correct(sentence):
    corrected = []
    similar =[]
   
    for i in sentence:
        similar.append(get_candidates(i))
    for j in similar:
        if len(j)!=1:
            for i in j:
                low = {}
                low[i] = prob(i)
            values = max(low.values())
            result = [key for key in low if low[key]==values]
            corrected.extend(result)
        else:
            corrected.extend(j)
     
    return corrected
correct(['NIGEL','aa', 'February', '48'])

#assert(correct(["this","whitr","cat"]) = ['this','white','cat'])   

['JILL', 'at', 'Bernard', '40']

In [156]:
print(test_original)

NameError: name 'test_original' is not defined

## **Task 5** (10 Marks): 
Using the test corpus evaluate the *accuracy* of your method, i.e., how many words from your system's output match the corrected sentence (you should count words that are already spelled correctly and not changed by the system).

In [165]:
from difflib import SequenceMatcher
def accuracy():
    test = data[:100]
    algo = []
    acc = []
    test_corrected = [i['corrected'] for i in test]
    test_original = [i['original'] for i in test]
    
    for i in range(0,len(test_original)):
        algo.append(correct(test_original[i]))
        acc.append(SequenceMatcher(None,test_corrected[i],algo[i]).ratio())
    total_accuracy = ((sum(acc)/len(acc))*100)
    return total_accuracy
accuracy()
#print(accuracy(test))

79.39544221449117

## **Task 6 (35 Marks):**

Consider a modification to your algorithm that would improve the accuracy of the algorithm developed in Task 3 and 4

* You may resources beyond those provided here.
* You must **not use the test data** in this task.
* Provide a short text describing what you intend to do and why. 
* Full marks for this section may be obtained without an implementation, but an implementation is preferred.
* Your implementation should not consist of more than 50 lines of code

Please note this task is marked according to: demonstration of knowledge from the lectures (10), originality and appropriateness of solution (10), completeness of description (10) and technical correctness (5)


## Solution

### Consider the following example for first three solutions:
    ['NIGEL','THRUSH', 'February', '48'] -->  ['JILL', 'JUST', 'Bernard', '40']
1. Named Entitiy - 
    In the algorithm the named entities such as names and organizational names were also condidered as a part of change which was not to be implemented like that. So all named entity should be handled.

2. Numerical Data -
    The numerical data was also changed so this should also be handled.ex - 

3. Dates - 
    For months like 'February' it was changed to 'Bernard' so this should be handled properly.

4. Increase in DataSet - 
    The training data was very less if it would have beem more exhaustive then it might have added the dictionary list ,for correction in turn giving more accuracy.

5. Morphology - 
    There may be cases where the word is not in its reduced form so we can use stemming and lemmatization of such words, basically to have normalized corpus.

6. Smoothing - 
    We can use Language Model Smoothing to reduce zero probabilities

7. Additional Methods - 
    We can further implement Hidden Markov Models to increase probability and better predict the output.
    

## **Task 7 (5 Marks):**

Repeat the evaluation (as in Task 5) of your new algorithm and show that it outperforms the algorithm from Task 3 and 4